In [2]:
import os
import csv
import tiktoken
import pandas as pd
from tqdm import tqdm
from nltk.tokenize import word_tokenize, sent_tokenize

In [3]:
enc = tiktoken.get_encoding("o200k_base") #cl100k_base

In [4]:
def get_csv_paths(folder_path, recursive=False):
    if recursive:
        # Walk through all subdirectories
        file_paths = [os.path.join(root, file) 
                      for root, _, files in os.walk(folder_path) 
                      for file in files if file.endswith('.csv')]
    else:
        # Get files in the root folder only
        file_paths = [os.path.join(folder_path, file) 
                      for file in os.listdir(folder_path) 
                      if file.endswith('.csv')]
    
    return file_paths

In [ ]:
def calc_stats(texts):
    results = []
    total_sentences, total_words, total_chars, total_tokens = 0, 0, 0, 0
    total_samples = len(texts)

    for text in tqdm(texts):
        text_chars = 0
        text_tokens = enc.encode(text)
        sentences = sent_tokenize(text)
        for sentence in sentences:
            words = word_tokenize(sentence)
            text_chars += sum([len(word) for word in words])

        total_sentences += len(sentences)
        total_words += len(words)
        total_chars += text_chars
        total_tokens += len(text_tokens)

        results.append([len(sentences), len(words), text_chars, len(text_tokens)])
    return results, total_samples, total_sentences, total_words, total_chars, total_tokens

In [ ]:
DATA_HUMAN_PATH = "../data/data_human"
DATA_AI_PATH = "../data/data_ai"
STATS_PATH = "../data/stats/"
MASTER_STATS_PATH = "../data/stats/data_stats_master.csv"

In [7]:
paths = get_csv_paths(DATA_HUMAN_PATH) + get_csv_paths(DATA_AI_PATH, recursive=True)

In [ ]:
with open(MASTER_STATS_PATH, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["data", "model", "num_samples", "num_sentences", "num_words", "num_chars", "num_tokens"])

for path in paths:
    if path.split("_")[-1] == "human.csv":
        stats_path = os.path.join(STATS_PATH, path.split("/")[-2], path.split("/")[-1].replace(".csv", "_features.csv"))
    else:
        stats_path = os.path.join(STATS_PATH, path.split("/")[-3], path.split("/")[-2], path.split("/")[-1].replace(".csv", "_features.csv"))


    df = pd.read_csv(path)
    texts = df["text"].values

    results, num_samples, num_sentences, num_words, num_chars, num_tokens = calc_stats(texts)

    results = pd.DataFrame(results, columns=["num_sentences", "num_words", "num_chars", "num_tokens"])
    results.to_csv(stats_path, index=False)

    data_name, model = path.split("/")[-1].split("_")
    model = model.removesuffix(".csv")

    with open(MASTER_STATS_PATH, mode="a", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow([data_name, model, num_samples, num_sentences, num_words, num_chars, num_tokens])

100%|██████████| 24/24 [00:00<00:00, 409.50it/s]
